In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import matplotlib.pyplot as plt
import mplleaflet
import numpy as np
import pandas as pd
from pandas.io.json import json_normalize
import requests
import json
from pprint import pprint

Note the naming convention for dataframes read from txt files are prefixed with gt_ as in google_transit

## Bus Stops

In [123]:
gt_stops = pd.read_csv('google_transit/stops.txt', sep=",")
gt_stops = gt_stops.drop(['stop_code','stop_url', 'stop_desc', 'zone_id', 'stop_timezone', 'wheelchair_boarding'], axis=1)

# This will treat all non numeric-looking strings as NaN
gt_stops['stop_id'] = gt_stops[gt_stops['stop_id'].str.isnumeric()==True]

# Drop all NaN rows
gt_stops.dropna(subset=['stop_id'], inplace=True)

# Cast numeric strings to int64
gt_stops['stop_id'] = gt_stops['stop_id'].astype('int64')

gt_stops

,stop_id,stop_name,stop_lat,stop_lon,location_type,parent_station
0,20,Kennedy Rd S n/of First Gulf Blvd,43.673256,-79.718468,0,NaN
1,30,Kennedy Rd S at Steeles Ave E,43.675159,-79.721710,0,NaN
2,55,Rutherford Rd S n/of Steeles Ave E,43.681385,-79.718147,0,NaN
3,60,Rutherford Rd S/of Bramsteele Rd,43.681969,-79.718979,0,NaN
4,70,Rutherford Rd S n/of Bramsteele Rd,43.683807,-79.721458,0,NaN
...,...,...,...,...,...,...
2946,19090339,Mississauga Rd s/of Royal West Dr,43.662159,-79.822662,0,NaN
2947,19102800,Castlemore Rd e/of McVean Dr,43.782913,-79.700104,0,NaN
2948,20030200,Van Kirk Dr n/of Whitepoppy Dr,43.724243,-79.829727,0,NaN
2949,20030201,Van Kirk Dr n/of Bramtrail Gate,43.720882,-79.824982,0,NaN


In [4]:
# gt_routes.loc[gt_routes['route_type'] == 3]
# gt_routes['route_long_name'].unique()

## Bus Routes

In [5]:
gt_routes = pd.read_csv('google_transit/routes.txt', sep=",")
gt_routes = gt_routes.drop(['route_url','route_desc','route_type'], axis=1) 
gt_routes

,route_id,route_short_name,route_long_name
0,1-274,1,Queen
1,2-274,2,Main
2,3-274,3,McLaughlin
3,4-274,4,Chinguacousy
4,5-274,5,Bovaird
...,...,...,...
68,501-274,501,Zum Queen
69,502-274,502,Zum Main
70,505-274,505,Zum Bovaird
71,511-274,511,Zum Steeles


## Calendar & Stop Times & Trips

In [6]:
gt_calendar = pd.read_csv('google_transit/calendar.txt', sep=",")
gt_calendar

,service_id,monday,tuesday,wednesday,thursday,friday,saturday,sunday,start_date,end_date
0,200302-MULTI-Weekday-01,1,1,1,1,1,0,0,20200302,20200424
1,200302-MULTI-Weekday-01-0001000,0,0,0,1,0,0,0,20200302,20200424
2,200302-MULTI-Weekday-01-0010000,0,0,1,0,0,0,0,20200302,20200424
3,200302-MULTI-Weekday-01-1000000,1,0,0,0,0,0,0,20200302,20200424
4,200302-MULTI-Saturday-01,0,0,0,0,0,1,0,20200307,20200425
5,200302-MULTI-Sunday-01,0,0,0,0,0,0,1,20200308,20200426
6,200302-MULTI-Holiday1-01,1,1,1,1,1,1,1,20200410,20200410


In [7]:
gt_stop_times = pd.read_csv('google_transit/stop_times.txt', sep=",")
gt_stop_times = gt_stop_times.drop(['timepoint', 'drop_off_type', 'pickup_type'], axis=1)
gt_stop_times

,trip_id,arrival_time,departure_time,stop_id,stop_sequence
0,10445576-200302-MULTI-Weekday-01,05:10:00,05:10:00,45565,1
1,10445576-200302-MULTI-Weekday-01,05:12:00,05:12:00,5260,2
2,10445576-200302-MULTI-Weekday-01,05:13:00,05:13:00,5270,3
3,10445576-200302-MULTI-Weekday-01,05:14:00,05:14:00,5280,4
4,10445576-200302-MULTI-Weekday-01,05:14:00,05:14:00,5290,5
...,...,...,...,...,...
486660,10528257-200302-MULTI-Holiday1-01,16:01:00,16:01:00,51112,18
486661,10528257-200302-MULTI-Holiday1-01,16:03:00,16:03:00,51114,19
486662,10528257-200302-MULTI-Holiday1-01,16:05:00,16:05:00,51116,20
486663,10528257-200302-MULTI-Holiday1-01,16:08:00,16:08:00,18030502,21


In [8]:
gt_trips = pd.read_csv('google_transit/trips.txt', sep=",")
gt_trips = gt_trips.drop(['wheelchair_accessible', 'bikes_allowed'], axis=1)
gt_trips

,route_id,service_id,trip_id,trip_headsign,direction_id,block_id,shape_id
0,17-274,200302-MULTI-Weekday-01,10445576-200302-MULTI-Weekday-01,17 HOWDEN NORTH,0,710327,170018
1,17-274,200302-MULTI-Weekday-01,10445577-200302-MULTI-Weekday-01,17 HOWDEN NORTH,0,710328,170018
2,17-274,200302-MULTI-Weekday-01,10445578-200302-MULTI-Weekday-01,17 HOWDEN NORTH,0,710327,170018
3,17-274,200302-MULTI-Weekday-01,10445579-200302-MULTI-Weekday-01,17 HOWDEN NORTH,0,710328,170018
4,17-274,200302-MULTI-Weekday-01,10445580-200302-MULTI-Weekday-01,17 HOWDEN NORTH,0,710327,170018
...,...,...,...,...,...,...,...
12723,11-274,200302-MULTI-Holiday1-01,10528241-200302-MULTI-Holiday1-01,11 STEELES WESTBOUND,1,712237,110073
12724,511-274,200302-MULTI-Holiday1-01,10528254-200302-MULTI-Holiday1-01,511 ZUM STEELES EASTBOUND,0,712242,5110017
12725,511-274,200302-MULTI-Holiday1-01,10528255-200302-MULTI-Holiday1-01,511 ZUM STEELES EASTBOUND,0,712239,5110017
12726,511-274,200302-MULTI-Holiday1-01,10528256-200302-MULTI-Holiday1-01,511 ZUM STEELES EASTBOUND,0,712240,5110017


In [203]:
# This will be the central dataframe
routes_master = pd.merge(gt_trips[['route_id', 'trip_id', 'direction_id']], gt_stop_times[['trip_id', 'stop_id', 'stop_sequence']], how='inner')

routes_master = pd.merge(routes_master, gt_stops[['stop_id', 'stop_name', 'stop_lon', 'stop_lat']], how='left')

routes_master.drop(columns=['trip_id'], inplace=True)
routes_master.drop_duplicates(inplace=True)

#routes_master.drop_duplicates(subset=['route_id', 'direction_id', 'stop_id', 'stop_sequence', 'stop_name'],inplace=True)
#routes_master.drop(columns=['trip_id'], inplace=True)

# Only use 1 direction
# routes_master[routes_master['direction_id'] == 0]
routes_master

,route_id,direction_id,stop_id,stop_sequence,stop_name,stop_lon,stop_lat
0,17-274,0,45565,1,Bramalea Terminal Route 9 EB/17/207 Stop,-79.720367,43.718792
1,17-274,0,5260,2,Hanover Rd w/of Hartnell Sq,-79.724983,43.720787
2,17-274,0,5270,3,Hanover Rd at Helena Crt,-79.728081,43.718014
3,17-274,0,5280,4,Hanover Rd opp Hasting Square,-79.729599,43.718796
4,17-274,0,5290,5,Hanover Rd s/of Howden Blvd,-79.731422,43.720169
...,...,...,...,...,...,...,...
386040,50-274,0,27010,10,The Gore Rd n/of Castle Oaks Crossing,-79.683411,43.787437
386041,50-274,0,27012,11,The Gore Rd opp Fitzpatrick Dr,-79.684998,43.788586
386042,50-274,0,27013,12,The Gore Rd opp Castlemore School,-79.686859,43.789932
386043,50-274,0,27020,13,The Gore Rd s/of Castlemore Rd,-79.689369,43.791809


## Create Dictionaries

In [128]:
stop_dict = gt_stops.to_dict(orient='records')
stop_dict[:2]

[{'stop_id': 20,
  'stop_name': 'Kennedy Rd S n/of First Gulf Blvd',
  'stop_lat': 43.673256,
  'stop_lon': -79.718468,
  'location_type': 0,
  'parent_station': nan},
 {'stop_id': 30,
  'stop_name': 'Kennedy Rd S at Steeles Ave E',
  'stop_lat': 43.675159,
  'stop_lon': -79.72171,
  'location_type': 0,
  'parent_station': nan}]

In [129]:
route_dict = gt_routes.to_dict(orient='records')
route_dict[:2]

[{'route_id': '1-274', 'route_short_name': 1, 'route_long_name': 'Queen'},
 {'route_id': '2-274', 'route_short_name': 2, 'route_long_name': 'Main'}]

In [130]:
stop_times_dict = gt_stop_times.to_dict(orient='records')
stop_times_dict[:2]

[{'trip_id': '10445576-200302-MULTI-Weekday-01',
  'arrival_time': '05:10:00',
  'departure_time': '05:10:00',
  'stop_id': 45565,
  'stop_sequence': 1},
 {'trip_id': '10445576-200302-MULTI-Weekday-01',
  'arrival_time': '05:12:00',
  'departure_time': '05:12:00',
  'stop_id': 5260,
  'stop_sequence': 2}]

In [131]:
trips_dict = gt_trips.to_dict(orient='records')
trips_dict[:2]

[{'route_id': '17-274',
  'service_id': '200302-MULTI-Weekday-01',
  'trip_id': '10445576-200302-MULTI-Weekday-01',
  'trip_headsign': '17 HOWDEN NORTH',
  'direction_id': 0,
  'block_id': 710327,
  'shape_id': 170018},
 {'route_id': '17-274',
  'service_id': '200302-MULTI-Weekday-01',
  'trip_id': '10445577-200302-MULTI-Weekday-01',
  'trip_headsign': '17 HOWDEN NORTH',
  'direction_id': 0,
  'block_id': 710328,
  'shape_id': 170018}]

In [194]:
# If we try querying via the route id 17-274, 
#    we can see that stops and names associated with both directions
test = routes_master.loc[(routes_master['route_id']=='17-274') & (routes_master['direction_id']==0),:][:50]
test

,route_id,trip_id,direction_id,stop_id,stop_sequence,stop_name,stop_lon,stop_lat
0,17-274,10445576-200302-MULTI-Weekday-01,0,45565,1,Bramalea Terminal Route 9 EB/17/207 Stop,-79.720367,43.718792
1,17-274,10445576-200302-MULTI-Weekday-01,0,5260,2,Hanover Rd w/of Hartnell Sq,-79.724983,43.720787
2,17-274,10445576-200302-MULTI-Weekday-01,0,5270,3,Hanover Rd at Helena Crt,-79.728081,43.718014
3,17-274,10445576-200302-MULTI-Weekday-01,0,5280,4,Hanover Rd opp Hasting Square,-79.729599,43.718796
4,17-274,10445576-200302-MULTI-Weekday-01,0,5290,5,Hanover Rd s/of Howden Blvd,-79.731422,43.720169
5,17-274,10445576-200302-MULTI-Weekday-01,0,5300,6,Howden Blvd at Dixie Rd,-79.733772,43.719582
6,17-274,10445576-200302-MULTI-Weekday-01,0,5310,7,Howden Blvd at Leander St,-79.737694,43.718353
7,17-274,10445576-200302-MULTI-Weekday-01,0,5320,8,Howden Blvd at Vodden Rd E,-79.741463,43.717125
8,17-274,10445576-200302-MULTI-Weekday-01,0,5330,9,Howden Blvd at Ladin Dr,-79.744026,43.718193
9,17-274,10445576-200302-MULTI-Weekday-01,0,5340,10,Howden Blvd at Leander St,-79.745491,43.719364


In [204]:
# Plot the routes
fig_routes, ax_routes = plt.subplots()

ax_routes.plot(test['stop_lon'],  test['stop_lat'], '.r')

# Display Inline
mplleaflet.display(fig=fig_routes)